In [ ]:
import pandas as pd
from pathlib import Path
from xgboost import XGBClassifier
from sklearn.model_selection import cross_validate
import warnings

In [ ]:
df_train = pd.read_csv('../input/tabular-playground-series-sep-2021/train.csv',
                      index_col = 'id',
                      #nrows = 25000
                      )
features = df_train.columns[:-1] 
target = df_train.columns[-1]

df_train.head()


In [ ]:
train = df_train.loc[:, features]
y = df_train.loc[:, target]

model = XGBClassifier(max_depth=3, 
                     subsample=0.5,
                     colsample_bytree=0.5,
                     n_jobs=-1,
                     tree_method='gpu_hist',
                     random_state=0)
def score(X, y, model, cv):
    scoring = ['roc_auc'] #AUC (area under curve) evaluation metric
    scores = cross_validate(model, X, y,
                           scoring=scoring,
                           cv=cv, return_train_score=True
                           )
    scores = pd.DataFrame(scores).T
    return scores.assign(mean=lambda x: x.mean(axis=1),
                        std=lambda x: x.std(axis=1),
                        )
scores = score(train, y, model, cv = 3)
display(scores)

In [ ]:
train.shape, y.shape

In [ ]:
model.fit(train, y)
xtest = pd.read_csv('../input/tabular-playground-series-sep-2021/test.csv',
                    index_col='id')

y_pred = pd.Series(model.predict(xtest),
                 index=xtest.index,
                 name=target,
                 )
y_pred.to_csv('submission.csv')